In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from upper_ocean_metrics.upper_ocean_metrics import ohc_from_profile, mld_temp_crit, mld_dens_crit, temp_average_depth, potential_energy_anomaly100
from hurricanes.calc import convert_lon_180_to_360, find_nearest
import seawater
from glob import glob
import os
from pathlib import Path

In [3]:
# Set main path of data and plot location
root_dir = Path.home() / "Documents"

# Paths to data sources
path_data = (root_dir / "data") # create data path
path_gliders = (path_data / "gliders")
# path_impact = (path_data / "impact_metrics")
# path_impact_calculated = path_impact / "calculated"
# path_impact_model = path_impact / "models"

In [4]:
glider = "ng645-20210613T0000"

# Read glider dataframe output from erddap
glider_pickle = path_gliders / f"{glider}_data.pkl"

try:
    df = pd.read_pickle(glider_pickle)
except FileNotFoundError:
    # Download glider data from erddap with dataset id
    df = get_glider_by_id(glider)
    df.to_pickle(glider_pickle) # Save glider data to pickle file
    

In [5]:
df

,longitude (degrees_east),latitude (degrees_north),pressure (decibar),depth (m),temperature (degrees_C),salinity (1),conductivity (mS cm-1)
time (UTC),,,,,,,
2021-06-13 04:26:50,-94.591942,27.643408,0.279798,0.277945,29.0704,31.186760,51.763603
2021-06-13 04:26:50,-94.591942,27.643408,0.189863,0.188605,29.0598,31.188139,51.755400
2021-06-13 04:26:50,-94.591942,27.643408,0.139899,0.138972,29.0657,31.188772,51.762000
2021-06-13 04:26:50,-94.591942,27.643408,0.189863,0.188605,29.0679,31.189420,51.765100
2021-06-13 04:26:50,-94.591942,27.643408,0.259812,0.258092,29.0700,31.188171,51.765297
...,...,...,...,...,...,...,...
2021-09-24 14:39:05,-88.167206,29.205790,192.345050,190.959230,15.1129,36.032272,44.243896
2021-09-24 14:39:05,-88.167206,29.205790,192.963730,191.573170,15.0898,36.028183,44.216400
2021-09-24 14:39:05,-88.167206,29.205790,193.612320,192.216780,15.0932,36.028347,44.220300


In [6]:
df = df.reset_index()
df = df.rename({"time (UTC)": "time", "longitude (degrees_east)": "lon", "latitude (degrees_north)": "lat"}, axis=1)
df = df.set_index("time").sort_index()
df

,lon,lat,pressure (decibar),depth (m),temperature (degrees_C),salinity (1),conductivity (mS cm-1)
time,,,,,,,
2021-06-13 04:26:50,-94.591942,27.643408,0.279798,0.277945,29.0704,31.186760,51.763603
2021-06-13 04:26:50,-94.591942,27.643408,0.189863,0.188605,29.0598,31.188139,51.755400
2021-06-13 04:26:50,-94.591942,27.643408,0.139899,0.138972,29.0657,31.188772,51.762000
2021-06-13 04:26:50,-94.591942,27.643408,0.189863,0.188605,29.0679,31.189420,51.765100
2021-06-13 04:26:50,-94.591942,27.643408,0.259812,0.258092,29.0700,31.188171,51.765297
...,...,...,...,...,...,...,...
2021-09-24 14:39:05,-88.167206,29.205790,192.345050,190.959230,15.1129,36.032272,44.243896
2021-09-24 14:39:05,-88.167206,29.205790,192.963730,191.573170,15.0898,36.028183,44.216400
2021-09-24 14:39:05,-88.167206,29.205790,193.612320,192.216780,15.0932,36.028347,44.220300


In [7]:
t0 = pd.Timestamp(2021, 8, 18).strftime("%Y-%m-%d")
t1 = pd.Timestamp(2021, 8, 18).strftime("%Y-%m-%d")

# For profile diagnostic purposes
df = df.loc[pd.IndexSlice[t0:t1], :]

In [8]:
df

,lon,lat,pressure (decibar),depth (m),temperature (degrees_C),salinity (1),conductivity (mS cm-1)
time,,,,,,,
2021-08-18 01:02:19,-89.559296,28.013130,0.250208,0.248544,30.7405,33.899765,57.489700
2021-08-18 01:02:19,-89.559296,28.013130,0.220183,0.218719,30.7410,33.900227,57.490902
2021-08-18 01:02:19,-89.559296,28.013130,0.260216,0.258486,30.7438,33.900265,57.493900
2021-08-18 01:02:19,-89.559296,28.013130,0.230191,0.228660,30.7452,33.909462,57.509197
2021-08-18 01:02:19,-89.559296,28.013130,0.200166,0.198835,30.7582,33.908768,57.521700
...,...,...,...,...,...,...,...
2021-08-18 23:07:16,-89.469421,28.079018,144.885960,143.871080,17.3084,36.361725,46.832200
2021-08-18 23:07:16,-89.469421,28.079018,145.886950,144.864720,17.1653,36.362490,46.685900
2021-08-18 23:07:16,-89.469421,28.079018,146.928010,145.898120,17.1588,36.346690,46.661602


In [ ]:
def calculate_density(temperature, salinity, depth, lat):
    pressure = seawater.eos80.pres(depth, lat)
    density = seawater.eos80.dens(salinity, temperature, pressure)
    return density


def calculate_upper_ocean_metrics(time, temp, salinity, depth, lat, density=None):
    """_summary_

    Args:
        time (_type_): time
        temp (_type_): temperature (c)
        salinity (_type_): salinity (1)
        depth (_type_): depth (m)
        density (_type_, optional): _description_. Defaults to None.
        lat (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    if not isinstance(density, pd.Series):
        density = pd.Series(dtype=object)
    
        
    # Reference variables
    dtemp = 0.2
    reference_depth = 10
    drho = 0.125

    # Init empty dict 
    ddict = {}
     
    # Calculate density of gofs profile. Converting from depth (m) to pressure 
    # internally.
    if not density.any():
        density = calculate_density(temp, salinity, depth, lat)
    
    # Ocean heat content
    # Depth, temperature, density
    ohc = ohc_from_profile(depth, temp, density)
    ddict['ocean_heat_content'] = ohc

    # Mixed Layer Depth calculated from temperature
    # dtemp, reference depth, depth, and temperature
    depth_mld_t, temp_mld_t= mld_temp_crit(dtemp, reference_depth, depth, temp)
    ddict['mixed_layer_depth_from_temp'] = depth_mld_t
    ddict['mixed_layer_temp_from_temp'] = temp_mld_t

    # Mixed Layer Depth calculated from density
    # ddensity, reference depth, depth, temperature, and density
    depth_mld_d, temp_mld_d = mld_dens_crit(drho, reference_depth, depth, temp,  density)
    ddict['mixed_layer_depth_from_density'] = depth_mld_d
    ddict['mixed_layer_temp_from_density'] = temp_mld_d

    # Average temperature in the top 100m 
    # depth, temperature
    ddict['average_temp_mldt_to_100m'] = temp_average_depth(depth, temp, depth_range=[depth_mld_t,100])
    ddict['average_temp_mlds_to_100m'] = temp_average_depth(depth, temp, depth_range=[depth_mld_d,100])
    ddict['average_temp_000m_to_100m'] = temp_average_depth(depth, temp, depth_range=[0,100])
    ddict['average_temp_100m_to_200m'] = temp_average_depth(depth, temp, depth_range=[100,200])
    ddict['average_temp_200m_to_300m'] = temp_average_depth(depth, temp, depth_range=[200,300])
    ddict['average_temp_300m_to_400m'] = temp_average_depth(depth, temp, depth_range=[300,400])
    ddict['average_temp_400m_to_500m'] = temp_average_depth(depth, temp, depth_range=[400,500])
    ddict['average_temp_500m_to_600m'] = temp_average_depth(depth, temp, depth_range=[500,600])   
    ddict['average_temp_600m_to_700m'] = temp_average_depth(depth, temp, depth_range=[600,700])
    ddict['average_temp_700m_to_800m'] = temp_average_depth(depth, temp, depth_range=[700,800])
    ddict['average_temp_800m_to_900m'] = temp_average_depth(depth, temp, depth_range=[800,900])
    ddict['average_temp_900m_to_1000m'] = temp_average_depth(depth, temp, depth_range=[900,1000])


    # Potential Energy Anomaly in the top 100 meters
    # depth, density
    pea = potential_energy_anomaly100(depth, density)
    ddict['potential_energy_anomaly_100m'] = pea

    # Salinity at surface
    # Should this be an average or the very first reading?
    sal_surf_idx = np.nanargmin(depth)
    ddict['salinity_surface'] = salinity[sal_surf_idx]

    # Salinity maximum
    try:
        sal_max_idx = np.nanargmax(salinity)
        ddict['salinity_max'] = salinity[sal_max_idx]
        ddict['salinity_max_depth'] = depth[sal_max_idx]
    except ValueError:
        ddict['salinity_max'] = np.nan
        ddict['salinity_max_depth'] = np.nan

    df = pd.DataFrame(data=ddict, index=[pd.to_datetime(time)])
    return df

In [ ]:

# # Glider - Iterate grouped glider times (each time is a profile)
# glider_df = pd.DataFrame()

# for time, group in df.groupby(level=0):
#     print(time)
#     gldf = calculate_upper_ocean_metrics(
#         time,
#         group['temperature (degrees_C)'].to_numpy(), 
#         group['salinity (1)'].to_numpy(), 
#         group['depth (m)'].to_numpy(), 
#         group['lat'].to_numpy(),
#         group['density (kg m-3)'].to_numpy()
#         )
#     glider_df = pd.concat([glider_df, gldf])
#     glider_df.to_csv(impact_calculated_dir / f"{glider}_calculated_glider_data.csv")
# # glider_df.to_pickle(impact_calculated_dir / f"{glider}_calculated_glider_data.pkl")